In [1]:
import json
import os
import gc
import numpy as np
import pandas as pd

def downsampling(positive_idx, negative_idx, ratio):
    idx = np.random.choice(negative_idx, int(ratio*len(negative_idx)))
    idx = np.concatenate((idx, positive_idx))
    return np.sort(idx).astype(int).tolist()


PATH1 = '/home/kai/data/kaggle/talkingdata/data/'
PATH0 = '/home/kai/data/kaggle/talkingdata/haoyandata1/'
PATH = '/home/kai/data/kaggle/talkingdata/downsampling/'

file = {}
train = pd.read_feather(PATH0 + 'train_cleaned.ftr')
# train = pd.read_csv(PATH1 + 'train_cleaned.csv', nrows=10000)
val_idx = (train['day'] == 9) & (train['hour'] == 13) | (train['hour'] == 17) | (train['hour'] == 21)
file['val_idx'] = val_idx[val_idx].index.values.astype(int).tolist()


# Downsampling
train = train[~val_idx]
negative = list(train[train['is_attributed']==0].index.values)
positive = list(train[train['is_attributed']==1].index.values)
ratio = len(positive)/len(negative)
idx_list = []
for i in range(5):
    idx_list.append(downsampling(positive, negative, ratio))
    print(i)
file['downsampling'] = idx_list
with open(PATH + 'idx_file.json', 'w') as f:
    json.dump(file, f)
print('done')

0
1
2
3
4
done


In [2]:
import os
import gc
import json
import pandas as pd

PATH = '/home/kai/data/kaggle/talkingdata/haoyandata1/'
DOWNSAMPLING_PATH = '/home/kai/data/kaggle/talkingdata/downsampling/'
cols = ['os', 'hour', 'device', 'channel', 'app']
with open(DOWNSAMPLING_PATH + 'idx_file.json', 'r') as f:
    data = json.load(f)
    val_idx = data['val_idx']
    downsampling_idx = data['downsampling']

train = pd.read_feather(PATH + 'train_cleaned.ftr')[cols]
test = pd.read_feather(PATH + 'test_cleaned.ftr')[cols]
for file in os.listdir(PATH):
    tmp = file.split('.')
    if tmp[0] != 'train_cleaned' and tmp[0] != 'test_cleaned' and tmp[1] == 'ftr':
        ttmp = tmp[0].split('_')
        print(ttmp)
        print(file)
        if ttmp[0] == 'train':
            train['_'.join(ttmp[1:])] = pd.read_feather(PATH + file)
        else:
            test['_'.join(ttmp[2:])] = pd.read_feather(PATH + file)
        gc.collect()

['train', 'woe', 'app', 'device']
train_woe_app_device.ftr
